### 딥러닝 모델 구현하기
* tensorflow를 활용하여 Bike데이터를 활용하여 딥러닝 모델을 구현해 보았음.
* tensorflow 활용한 딥러닝 모델 구현
* tf version(1.14.0)
* 설치(로컬).CPU버전 : pip install tensorflow==1.14.0

#### colab에서 시작하기

<a href="https://colab.research.google.com/github/LDJWJ/00_TOTO_MLDL_CLASS/blob/master/DL02_04_TF_Bike.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import pandas as pd
import tensorflow as tf
import numpy as np

In [10]:
print(tf.__version__)
print(np.__version__)
print(pd.__version__)

1.15.0
1.18.1
0.25.3


In [11]:
# colab 에 환경에서 확인 가능
!ls

'ls'은(는) 내부 또는 외부 명령, 실행할 수 있는 프로그램, 또는
배치 파일이 아닙니다.


In [12]:
# colab 환경의 경우 데이터 셋 업로드 필요
train = pd.read_csv("./train_bike.csv", parse_dates=['datetime'])
test = pd.read_csv("./test_bike.csv", parse_dates=['datetime'])
sub = pd.read_csv("./sampleSubmission_bike.csv")

In [13]:
print(train.columns)
print(test.columns)
print(sub.columns)

print(train.shape)
print(test.shape)
print(sub.shape)

Index(['datetime', 'season', 'holiday', 'workingday', 'weather', 'temp',
       'atemp', 'humidity', 'windspeed', 'casual', 'registered', 'count'],
      dtype='object')
Index(['datetime', 'season', 'holiday', 'workingday', 'weather', 'temp',
       'atemp', 'humidity', 'windspeed'],
      dtype='object')
Index(['datetime', 'count'], dtype='object')
(10886, 12)
(6493, 9)
(6493, 2)


In [16]:
# input_col = [ 'temp', 'atemp']
# input(컬럼1개), target(컬럼1개)
input_col = ['temp']
labeled_col = ['count']

In [17]:
x_data = train[input_col]    # 입력 X
y_data = train[labeled_col]  # 출력 Y (target)
x_test = test[input_col]     # test을 위한 입력 X
print(type(x_data))

# 자료형을 numpy로 변환
x_data = np.array(x_data)   
x_test = np.array(x_test)
y_data = np.array(y_data)
print(type(x_data))

<class 'pandas.core.frame.DataFrame'>
<class 'numpy.ndarray'>


In [18]:
# weight와 bias 지정
W1 = tf.Variable(tf.random_uniform([1], -1.0, 1.0))
b = tf.Variable(tf.random_uniform([1], -1.0, 1.0))

In [19]:
# X의 값, Y의 값 플레이스홀더 지정
X1 = tf.placeholder(tf.float32, name="X1")
Y = tf.placeholder(tf.float32, name="Y")

In [20]:
# hypothesis = W1 * X1 + W2 * X2 + b
hypothesis = W1 * X1 + b

In [21]:
### 비용함수, 최적화 함수
cost = tf.reduce_mean(tf.square(hypothesis - Y))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
train_op = optimizer.minimize(cost)

### 학습하기

In [22]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for step in range(100):
        _, cost_val = sess.run([train_op, cost], feed_dict={X1:x_data, Y:y_data})
        
        if step%10==0:
            print(step, cost_val, sess.run(W1), sess.run(b))
            
        
    pred = sess.run(hypothesis, feed_dict={X1:x_test})
    
print(pred.shape)
print(type(pred))
print(pred[0:5])
pred = pred.reshape(-1,)
pred.shape

0 77376.39 [8.794364] [0.8694137]
10 27708.63 [9.391758] [0.9084262]
20 27708.617 [9.391189] [0.9216579]
30 27708.602 [9.390621] [0.9348556]
40 27708.584 [9.390053] [0.9480193]
50 27708.566 [9.389488] [0.96114916]
60 27708.543 [9.388925] [0.97424525]
70 27708.531 [9.388362] [0.9873074]
80 27708.514 [9.387801] [1.0003362]
90 27708.502 [9.387242] [1.0133313]
(6493, 1)
<class 'numpy.ndarray'>
[[101.08765]
 [101.08765]
 [101.08765]
 [101.08765]
 [101.08765]]


(6493,)

### 값을 예측하고, csv파일 만들기

In [23]:
dat = pd.DataFrame({'count':pred})
print(sub.columns)
sub['count'] = dat['count']
sub.to_csv("tf_pred.csv", index=False)

Index(['datetime', 'count'], dtype='object')


### 제출하고 확인해 보기

### 실습과제 : 변수를 두개를 가지고 수행해 보기

### 그렇다면 변수가 계속 늘어난다면 식을 하나 하나 구현해야만 할까?

### 행렬 연산을 활용한 딥러닝 모델 만들어보기

In [24]:
input_col = [ 'temp', 'atemp']
labeled_col = ['count']

x_data = train[input_col]
y_data = train[labeled_col]

x_test = test[input_col]

x_data = np.array(x_data)
y_data = np.array(y_data)
print(x_data.shape, y_data.shape)

(10886, 2) (10886, 1)


In [25]:
X1 = tf.placeholder(tf.float32, [None, 2], name="X1")
Y = tf.placeholder(tf.float32, [None, 1], name="Y")

W1 = tf.Variable(tf.random_uniform([2, 1], -1.0, 1.0))
b = tf.Variable(tf.random_uniform([1], -1.0, 1.0))

In [26]:
# X = tf.placeholder(tf.float32, [None, 2])   # [? * 2] X [2 * 1] => ? X 1 
# W = tf.Variable(tf.random_normal([2,1]))

hypothesis = tf.matmul(X1, W1) + b
hypothesis

<tf.Tensor 'add_1:0' shape=(?, 1) dtype=float32>

In [27]:
### 비용함수, 최적화 함수
cost = tf.reduce_mean(tf.square(hypothesis - Y))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.0001)
train_op = optimizer.minimize(cost)

In [28]:
%%time

sess = tf.Session()
sess.run(tf.global_variables_initializer())

for step in range(100):
    _, cost_val = sess.run([train_op, cost], feed_dict={X1:x_data, Y:y_data})
    
    if step%10==0:
        print(step, cost_val, sess.run(W1[0]), sess.run(W1[1]))

        
pred = sess.run(hypothesis, feed_dict={X1:x_test})
pred = pred.reshape(-1,)

sess.close()

0 75635.35 [1.3542566] [0.19988072]
10 28056.486 [4.4403777] [3.7634976]
20 27727.686 [4.7034364] [4.0540667]
30 27725.324 [4.7318287] [4.072593]
40 27725.219 [4.7406993] [4.06852]
50 27725.135 [4.7479324] [4.0625806]
60 27725.045 [4.755015] [4.0564995]
70 27724.957 [4.76207] [4.050419]
80 27724.873 [4.769109] [4.044351]
90 27724.781 [4.776131] [4.038296]
Wall time: 819 ms


In [40]:
## 행렬 이용 컬럼 2개(input) 536ms
## 행렬 사용 안함 컬럼 2개(input) : 1m 44s